<a href="https://colab.research.google.com/github/fabriziobasso/Colab_backup/blob/main/Chapter_IV_Baseline_Forecasts_using_NIXTLA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Chapter IV Baseline Forecasts using NIXTLA**

## 01 Importing holdout (test) and validation datasets

In [ ]:
%%capture
!pip install statsforecast==1.7.8
!pip install datasetsforecast==0.0.8

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

In [ ]:
%%capture
# Clone the repository
!git clone https://github.com/PacktPublishing/Modern-Time-Series-Forecasting-with-Python-2E.git

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import os
import plotly.io as pio
pio.templates.default = "plotly_white"
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
import missingno as msno
from itertools import cycle
from sklearn.metrics import mean_absolute_error
from IPython.display import display, HTML
# %load_ext autoreload
# %autoreload 2
np.random.seed()
tqdm.pandas()

# Navigate to the repository's root directory
%cd Modern-Time-Series-Forecasting-with-Python-2E

from src.utils.data_utils import compact_to_expanded
from src.imputation.interpolation import SeasonalInterpolation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd

In [ ]:
os.makedirs("/content/drive/MyDrive/Books/Modern Time Series Forecasting/Chapter II/imgs/chapter_2", exist_ok=True)
preprocessed = Path("/content/drive/MyDrive/Books/Modern Time Series Forecasting/Data/data/london_smart_meters/data/london_smart_meters/preprocessed")

In [ ]:
assert preprocessed.is_dir(), "You have to run 02 - Preprocessing London Smart Meter Dataset.ipynb in Chapter02 before running this notebook"

In [ ]:
def format_plot(fig, legends = None, font_size=15, title_font_size=20):
    if legends:
        names = cycle(legends)
        fig.for_each_trace(lambda t:  t.update(name = next(names)))
    fig.update_layout(
            autosize=False,
            width=900,
            height=500,
            title={
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            titlefont={
                "size": title_font_size
            },
            legend_title = None,
            legend=dict(
                font=dict(size=font_size),
                orientation="h",
                yanchor="bottom",
                y=0.98,
                xanchor="right",
                x=1,
            ),
            yaxis=dict(
                title_text="Value",
                titlefont=dict(size=font_size),
                tickfont=dict(size=font_size),
            ),
            xaxis=dict(
                title_text="Day",
                titlefont=dict(size=font_size),
                tickfont=dict(size=font_size),
            )
        )
    return fig

In [ ]:
import numpy as np
import pandas as pd
import time
import os
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from itertools import cycle

pio.templates.default = "plotly_white"
import warnings
import humanize

from functools import partial
from statsforecast.core import StatsForecast
from utilsforecast.plotting import plot_series
from utilsforecast.evaluation import evaluate
from statsforecast.models import (
    Naive,
    SeasonalNaive,
    HistoricAverage,
    WindowAverage,
    SeasonalWindowAverage,
    RandomWalkWithDrift,
    HoltWinters,
    ETS,
    AutoETS,
    AutoARIMA,
    ARIMA,
    AutoTheta,
    DynamicTheta,
    DynamicOptimizedTheta,
    Theta,
    OptimizedTheta,
    TBATS,
    AutoTBATS,
    MSTL

)
from datasetsforecast.losses import *
from src.utils.ts_utils import forecast_bias

import time
from src.utils import plotting_utils

from tqdm import tqdm
np.random.seed(42)
tqdm.pandas()

In [ ]:
import statsforecast as stf
stf.__version__

In [ ]:
# this makes it so that the outputs of the predict methods have the id as a column
# instead of as the index
os.environ['NIXTLA_ID_AS_COL'] = '1'

* **Set up Folders**

In [ ]:
os.makedirs("/content/drive/MyDrive/Books/Modern Time Series Forecasting/Chapter IV/chapter_4", exist_ok=True)
preprocessed = Path("/content/drive/MyDrive/Books/Modern Time Series Forecasting/Data/data/london_smart_meters/data/london_smart_meters/preprocessed")

* **Graph Functions and Formatting**

In [ ]:
def format_plot(fig, legends = None, xlabel="Time", ylabel="Value", title="", font_size=15):
    if legends:
        names = cycle(legends)
        fig.for_each_trace(lambda t:  t.update(name = next(names)))
    fig.update_layout(
            autosize=False,
            width=900,
            height=500,
            title_text=title,
            title={
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            titlefont={
                "size": 20
            },
            legend_title = None,
            legend=dict(
                font=dict(size=font_size),
                orientation="h",
                yanchor="bottom",
                y=0.98,
                xanchor="right",
                x=1,
            ),
            yaxis=dict(
                title_text=ylabel,
                titlefont=dict(size=font_size),
                tickfont=dict(size=font_size),
            ),
            xaxis=dict(
                title_text=xlabel,
                titlefont=dict(size=font_size),
                tickfont=dict(size=font_size),
            )
        )
    return fig

def plot_forecast(pred_df, forecast_columns, timestamp_col, forecast_display_names=None):
    if forecast_display_names is None:
        forecast_display_names = forecast_columns
    else:
        assert len(forecast_columns) == len(forecast_display_names)

    mask = ~pred_df[forecast_columns[0]].isnull()
    colors = [c.replace("rgb", "rgba").replace(")", ", <alpha>)") for c in px.colors.qualitative.Dark2]
    act_color = colors[0]
    colors = cycle(colors[1:])
    dash_types = cycle(["dash", "dot", "dashdot"])

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=pred_df[mask][timestamp_col], y=pred_df[mask]['energy_consumption'],
                             mode='lines', line=dict(color=act_color.replace("<alpha>", "0.3")),
                             name='Actual Consumption'))

    for col, display_col in zip(forecast_columns, forecast_display_names):
        fig.add_trace(go.Scatter(x=pred_df[mask][timestamp_col], y=pred_df.loc[mask, col],
                                 mode='lines', line=dict(dash=next(dash_types), color=next(colors).replace("<alpha>", "1")),
                                 name=display_col))
    return fig

* **Load the Datasets**

In [ ]:
#Readin the missing value imputed and train test split data
try:
    train_df = pd.read_parquet(preprocessed/"selected_blocks_train_missing_imputed.parquet")
    train_df = train_df[['LCLid',"timestamp","energy_consumption","frequency"]]
    val_df = pd.read_parquet(preprocessed/"selected_blocks_val_missing_imputed.parquet")
    val_df = val_df[['LCLid',"timestamp","energy_consumption","frequency"]]
    test_df = pd.read_parquet(preprocessed/"selected_blocks_test_missing_imputed.parquet")
    test_df = test_df[['LCLid',"timestamp","energy_consumption","frequency"]]
except FileNotFoundError:
    print(f"Warning: File not found in {preprocessed}. Ensure you've run '01-Setting up Experiment Harness.ipynb' in Chapter 04 and that the file path is correct.")

In [ ]:
train_df.shape, val_df.shape, test_df.shape

In [ ]:
print("Min train_df Date: " , train_df.timestamp.min())
print("Max train_df Date: " , train_df.timestamp.max())
print("Min val_df Date: " , val_df.timestamp.min())
print("Max val_df Date: " , val_df.timestamp.max())
print("Min test_df Date: " , test_df.timestamp.min())
print("Max test_df Date: " , test_df.timestamp.max())

In [ ]:
train_df = train_df[train_df.timestamp >'2012-01-01']
print("Min train_df Date: " , train_df.timestamp.min())

In [ ]:
train_df.shape

In [ ]:
#picking a single time series from the dataset for illustration
freq = train_df.iloc[0]['frequency']
ts_train = train_df.loc[train_df.LCLid=="MAC000193", ['LCLid',"timestamp","energy_consumption"]]
ts_val = val_df.loc[val_df.LCLid=="MAC000193", ['LCLid',"timestamp","energy_consumption"]]
ts_test = test_df.loc[test_df.LCLid=="MAC000193", ['LCLid',"timestamp","energy_consumption"]]

In [ ]:
ts_train.shape

## 02 Baseline Forecasts

### **Choosing an evaluation metric**

In machine learning, we have a handful of metrics that can be used to measure continuous outputs,
mainly Mean Absolute Error (MAE) and Mean Squared Error (MSE). But in the time series forecasting
realm, there are scores of metrics with no real consensus on which ones to use. One of the reasons for
this overwhelming number of metrics is that no one metric measures every characteristic of a forecast.
Therefore, we have a whole chapter devoted to this topic (Chapter 18, Evaluating Forecasts – Forecast
Metrics). For now, we will just review a few metrics, all of which we are going to use to measure the
forecasts. We are just going to consider them at face value:
* **Mean Absolute Error** (MAE): MAE is a very simple metric. It is the average of the unsigned
error between the forecast at timestep() and the observed value at time ().
Here, N is the number of time series, L is the length of time series (in this case, the length of
the test period), and f and y are the forecast and observed values, respectively.
* **Mean Squared Error** (MSE): MSE is the average of the squared error between the forecast ( )
and observed ( ).
* **Mean Absolute Scaled Error** (MASE): MASE is slightly more complicated than MSE or MAE
but gives us a slightly better measure to overcome the scale-dependent nature of the previous
two measures. If we have multiple time series with different average values, MAE and MSE
will show higher errors for the high-value time series as opposed to the low-valued time series.
MASE overcomes this by scaling the errors based on the in-sample MAE from the naïve
forecasting method (which is one of the most basic forecasts possible; we will review it later
in this chapter). Intuitively, MASE gives us the measure of how much better our forecast is as
compared to the naïve forecast:
* **Forecast Bias** (FB): This is a metric with slightly different aspects from the other metrics we’ve seen. While the other metrics help assess the correctness of the forecast, irrespective of the direction of the error, forecast bias lets us understand the overall bias in the model. Forecast
bias is a metric that helps us understand whether the forecast is continuously over- or under forecasting. We calculate forecast bias as the difference between the sum of the forecast and the
sum of the observed values, expressed as a percentage over the sum of all actuals:

Now, our test harness is ready. We also know how to evaluate and compare forecasts that have been
generated from different models on a single, fixed holdout dataset with a set of predetermined metrics. Now, it’s time to start forecasting.

In [ ]:
pred_df = pd.concat([ts_train, ts_val])

In [ ]:
def evaluate_performance(ts_train, ts_test, models, metrics, freq, level, id_col, time_col, target_col, h, metric_df=None):
    if metric_df is None:
        metric_df = pd.DataFrame()  # Initialize an empty DataFrame if not provided

    results = ts_test.copy()

    # Timing dictionary to store train and predict durations
    timing = {}

    for model in models:
        model_name = model.__class__.__name__
        evaluation = {}  # Reset the evaluation dictionary for each model

        # Start the timer for fitting and prediction
        start_time = time.time()

        # Instantiate StatsForecast class
        sf = StatsForecast(
            models=[model],
            freq=freq,
            n_jobs=-1,
            fallback_model=Naive()
        )

        # Efficiently predict without storing memory
        y_pred = sf.forecast(
            h=h,
            df=ts_train,
            id_col=id_col,
            time_col=time_col,
            target_col=target_col,
            level=level,
        )

        # Calculating the duration
        duration = time.time() - start_time
        timing[model_name] = duration

        # Merge prediction results to the original dataframe
        results = results.merge(y_pred, how='left', on=[id_col, time_col])

        ids = ts_train[id_col].unique()
        # Calculate metrics
        for id in ids:
            temp_results = results[results[id_col] == id]
            temp_train = ts_train[ts_train[id_col] == id]
            for metric in metrics:
                metric_name = metric.__name__
                #print(metric_name)
                if metric_name == 'mase':
                    evaluation[metric_name] = metric(temp_results[target_col].values,
                                                    temp_results[model_name].values,
                                                    temp_train[target_col].values, seasonality=48)
                #elif metric_name == 'smape':
                #    print(metric(temp_results[target_col].values, temp_results[model_name].values))

                else:
                    evaluation[metric_name] = metric(temp_results[target_col].values, temp_results[model_name].values)
            evaluation[id_col] = id
            evaluation['Time Elapsed'] = timing[model_name]

            # Prepare and append this model's results to metric_df
            temp_df = pd.DataFrame(evaluation, index=[0])
            temp_df['Model'] = model_name
            metric_df = pd.concat([metric_df, temp_df], ignore_index=True)

    return results, metric_df

### **Naïve forecast**

A naïve forecast is as simple as you can get. The forecast is just the last/most recent observation in a time series. If the latest observation in a time series is 10, then the forecast for all future timesteps is 10. This can be implemented as follows using the NaiveSeasonal class in darts:

In [ ]:
metrics = pd.DataFrame()

results, metrics = evaluate_performance(
                                        ts_train=ts_train,
                                        ts_test=ts_val,
                                        models=[Naive()],
                                        metrics=[mase, mae, mse, rmse, forecast_bias],
                                        freq=freq,
                                        level=[],  # Ensure this is correct or adjust as necessary
                                        id_col='LCLid',
                                        time_col='timestamp',
                                        target_col='energy_consumption',
                                        h=len(ts_val),
                                        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
                                    )

In [ ]:
metrics

In [ ]:
model_name = ['Naive']
model_display_name = ['Naive']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/naive.png")
fig.show()

### **Moving average forecast**
While a naïve forecast memorizes the most recent past, it also memorizes the noise at any timestep.
A moving average forecast is another simple method that tries to overcome the pure memorization
of the naïve method. Instead of taking the latest observation, it takes the mean of the latest n steps as the forecast. Moving average is not one of the models present in darts, but we have implemented a darts-compatible model in this book’s GitHub repository in the chapter04 folder:

In [ ]:
results, metrics = (
    evaluate_performance(
        ts_train,
        ts_val,
        models = [WindowAverage(window_size = 48)],
        metrics = [mase, mae, mse, rmse,forecast_bias],
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)

In [ ]:
metrics

In [ ]:
model_name = ['WindowAverage']
model_display_name = ['WindowAverage']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MSE: {metrics.loc[metrics.Model==model_name[0]][['mse']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/window_average.png")
fig.show()

### **Seasonal naive forecast**
A seasonal naive forecast is a twist on the simple naive method. Whereas in the naive method, we
took the last observation (Yt−1), in seasonal naïve, we take the Yt−k observation. So, we look back k steps for each forecast. This enables the algorithm to mimic the last seasonality cycle. For instance, if we set k=48*7, we will be able to mimic the latest seasonal weekly cycle. This method is implemented in darts and we can use it like so:

In [ ]:
results, metrics = (
                    evaluate_performance(
                        ts_train=ts_train,
                        ts_test=ts_val,
                        models = [SeasonalNaive(season_length=48*7)],
                        metrics = [mase, mae, mse, rmse, forecast_bias],
                        freq = freq,
                        level = [] ,
                        id_col = 'LCLid',
                        time_col = 'timestamp',
                        target_col = 'energy_consumption',
                        h = len(ts_val),
                        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
                        )
                    )

In [ ]:
results.head()

In [ ]:
metrics

In [ ]:
model_name = ['SeasonalNaive']
model_display_name = ['SeasonalNaive']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/seasonal_naive.png")
fig.show()

### **Exponential smoothing (ETS)**
Exponential smoothing (ETS) is one of the most popular methods for generating forecasts. It has been around since the late 1950s and has proved its mettle and stood the test of time. There are a few different variants of ETS – **single exponential smoothing**, **double exponential smoothing**, **Holt-Winters’ seasonal smoothing**, and so on. But all of them have one key idea that has been used in different ways.
In the naïve method, we were just using the latest observation, which is like saying only the most recent data point in history matters and no data point before that matters. On the other hand, the moving average method considers the last n observations to be equally important and takes the mean of them.
ETS combines both these intuitions and says that all the history is important, but the recent history is more important. Therefore, the forecast is generated using a weighted average where the weights decrease exponentially as we move farther into the history

In [ ]:
results, metrics = (
    evaluate_performance(
        ts_train,
        ts_val,
        models = [ HoltWinters(error_type = 'A', season_length = 48)],
        metrics = [mase, mae, mse, rmse,forecast_bias],
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)

In [ ]:
metrics

In [ ]:
model_name = ['HoltWinters']
model_display_name = ['HoltWinters']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MSE: {metrics.loc[metrics.Model==model_name[0]][['mse']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/ets.png")
fig.show()

In [ ]:
results, metrics = (
    evaluate_performance(
        ts_train,
        ts_val,
        models = [ AutoETS(model = 'AAA',season_length = 48)],
        metrics = [mase, mae, mse, rmse,forecast_bias],
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)

In [ ]:
metrics

In [ ]:
model_name = ['AutoETS']
model_display_name = ['AutoETS']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MSE: {metrics.loc[metrics.Model==model_name[0]][['mse']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/ets.png")
fig.show()

### **ARIMA**
Autoregressive Integrated Moving Average (ARIMA) models are the other class of methods that, like
ETS, have stood the test of time and are one of the most popular classical methods of forecasting. The
ETS family of methods is modeled around trend and seasonality, while ARIMA relies on autocorrelation
(the correlation of with −1, −2, and so on)

In [ ]:
results, metrics = (
    evaluate_performance(
        ts_train,
        ts_val,
        models = [ ARIMA(order = (2,1,1), seasonal_order = (1,1,1), season_length = 48)],
#        models = [ ARIMA(order = (0,1,2), seasonal_order = (0,0,2), season_length = 48)],
        metrics = [mase, mae, mse, rmse, forecast_bias],
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)

In [ ]:
metrics

In [ ]:
metrics

In [ ]:
model_name = ['ARIMA']
model_display_name = ['ARIMA']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MSE: {metrics.loc[metrics.Model==model_name[0]][['mse']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/ARIMA.png")
fig.show()

### **Sample Auto ARIMA**

Note: This may take a long time to run.

To get a better understanding of what parameters to use, you can run AutoARIMA and output the fitted parameters. AutoARIMA however is very slow, so I have includedd a sample output from AutoARIMA.

'arma': (0, 2, 0, 2, 48, 1, 0)

In [ ]:
# sf = StatsForecast(
#     models=[AutoARIMA( max_p = 2, max_d=1, max_q = 2, max_P=2, max_D = 1, max_Q = 2,
#                       start_p = 1, start_q = 1, start_P = 1, start_Q = 1, stepwise = True, season_length=48)],
#     freq=freq,
#     n_jobs=-1,
#     fallback_model = Naive()
# )

# y_pred = sf.fit(

#                     df=ts_train,
#                     id_col = 'LCLid',
#                     time_col = 'timestamp',
#                     target_col = 'energy_consumption',

#                     )

# sf.fitted_[0,0].model_

### **Theta Forecast**
The Theta Forecast was the top-performing submission in the M3 forecasting competition that was
held in 2002. The method relies on a parameter, θ, that amplifies or smooths the local curvature of a time series, depending on the value chosen. Using θ, we smooth or amplify the original time series.

These smoothed lines are called theta lines. V. Assimakopoulos and K. Nikolopoulos proposed this
method as a decomposition approach to forecasting. Although in theory any number of theta lines
can be used, the originally proposed method used two theta lines, =0 and =2, and took an average
of the forecast of the two theta lines as the final forecast.

In [ ]:
results, metrics = (
    evaluate_performance(
        ts_train,
        ts_val,
        models = [ Theta(season_length =48, decomposition_type = 'additive' )],
        metrics = [mase, mae, mse, rmse, forecast_bias],
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)

In [ ]:
metrics

In [ ]:
model_name = ['Theta']
model_display_name = ['Theta']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MSE: {metrics.loc[metrics.Model==model_name[0]][['mse']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/auto_theta.png")
fig.show()

### **TBATS**

NIXTLA again offers TBATS and AutoTBATS. AutoTBATS can be slow, so here we will just do TBATS. Similar to AutoARIMA, you can run a model using AutoTBATS to get a recommendation of good parameters to use.

In [ ]:
sf = StatsForecast(
    models=[TBATS(seasonal_periods  = 48, use_trend=True, use_damped_trend=True)],
    freq=freq,
    n_jobs=2
)

y_pred = sf.fit(

                    df=ts_train,
                    id_col = 'LCLid',
                    time_col = 'timestamp',
                    target_col = 'energy_consumption',

                    )
#sf.fitted_[0,0].model_

In [ ]:
results, metrics = (
    evaluate_performance(
        ts_train,
        ts_val,
        models = [ TBATS(seasonal_periods = [48])],
        metrics = [mase, mae, mse, rmse,forecast_bias],
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)

In [ ]:
metrics

### **MSTL**

In [ ]:
results, metrics = (
    evaluate_performance(
        ts_train,
        ts_val,
        models = [ MSTL(season_length = 48)],
        metrics = [mase, mae, mse, rmse, forecast_bias],
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)

In [ ]:
model_name = ['MSTL']
model_display_name = ['MSTL']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MSE: {metrics.loc[metrics.Model==model_name[0]][['mse']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/auto_tbats.png")
fig.show()

In [ ]:
metric_styled = metrics.reset_index(drop=True).style.format({
            "mae": "{:.3f}",
            "mse": "{:.3f}",
            "mase": "{:.3f}",
            "rmse": "{:.3f}",
            "forecast_bias": "{:.2f}%"}).highlight_min(color='lightgreen', subset=["mae","mse","mase","rmse","Time Elapsed"])
display(metric_styled)

Based on this sample test, the best performing models are (HoltWinters, and ETS), ARIMA, and TBATS. Lets build that for all models using AutoETS and TBATS. ARIMA gives similar performance to TBATS, but TBATS is faster.